## imports

In [365]:
import pandas as pd
import datetime

In [366]:
def flatten_aggregated_dataframe(
    gdf, concat_name=True, concat_separator=' ', name_level=1, inplace=False):
    """
    Flatten aggregated DataFrame.

    Args:
        gdf: DataFrame obtained through aggregation.
        concat_name: Whether to concatenate original column name and
            aggregation function name in the case of MultiIndex columns.
        concat_separator: Which string to place between original column name
            and aggregation function name if concat_name is True.
        name_level: Which element of a column tuple to use in the case of 
            MultiIndex columns and concat_name == False. Should be 0 for 
            original column name and 1 for aggregation function name.
        inplace: Whether to modify the aggregated DataFrame directly 
            (or return a copy).
    """
    if not inplace:
        gdf = gdf.copy()
    if type(gdf.columns) == pd.core.index.MultiIndex:
        if concat_name:
            columns = [concat_separator.join(col) for col in gdf.columns]
        else:
            columns = [col[name_level % 2] for col in gdf.columns]
        gdf.columns = columns
    return gdf.reset_index()

## create description file

## loading csv files

In [367]:
list_ = []

df = pd.read_csv("joc.csv", skiprows=1, index_col=None)
df.insert(0, "institution", "joc") 
df.insert(1, "typeOfInstitution", "museum") 
list_.append(df)
df = pd.read_csv("llem.csv", skiprows=1, index_col=None)
df.insert(0, "institution", "llem") 
df.insert(1, "typeOfInstitution", "other") 
list_.append(df)
df = pd.read_csv("mdc.csv", skiprows=1, index_col=None)
df.insert(0, "institution", "mdc") 
df.insert(1, "typeOfInstitution", "museum") 
list_.append(df)
df = pd.read_csv("mlel.csv", skiprows=1, index_col=None)
df.insert(0, "institution", "mlel") 
df.insert(1, "typeOfInstitution", "other") 
list_.append(df)
df = pd.read_csv("spl.csv", skiprows=1, index_col=None)
df.insert(0, "institution", "spl") 
df.insert(1, "typeOfInstitution", "library") 
list_.append(df)
df = pd.read_csv("ysl.csv", skiprows=1, index_col=None)
df.insert(0, "institution", "ysl") 
df.insert(1, "typeOfInstitution", "museum") 
list_.append(df)

entries = pd.concat(list_, axis = 0, ignore_index = True)

entries = entries.rename(index=str, columns={"Date": "date", "Entrees": "entrees"})

In [368]:
entries.head()

,institution,typeOfInstitution,date,entrees
0,joc,museum,01/11/2018 00:00,0
1,joc,museum,01/11/2018 00:30,0
2,joc,museum,01/11/2018 01:00,0
3,joc,museum,01/11/2018 01:30,0
4,joc,museum,01/11/2018 02:00,0


## re-structuring table

In [369]:
entries.insert(2, "time", "") 

In [370]:
entries['date'], entries['time'] = entries['date'].str.split(' ', 1).str
#entries['date'] = pd.to_datetime(entries.date)

### Aggregating by day

In [371]:
entries = entries.groupby(['institution','date']).sum()
entries = flatten_aggregated_dataframe(entries)
#entries = drop
entries

,institution,date,entrees
0,joc,01/01/2019,0
1,joc,01/11/2018,35962
2,joc,01/12/2018,18081
3,joc,02/01/2019,46300
4,joc,02/11/2018,41984
5,joc,02/12/2018,29390
6,joc,03/01/2019,32346
7,joc,03/11/2018,29930
8,joc,03/12/2018,23393
9,joc,04/01/2019,36077


In [372]:
entries['day'], entries['month'], entries['year'] = entries['date'].str.split('/').str

In [373]:
entries['week'] = pd.to_datetime(entries['date'], format='%d/%m/%Y').dt.week

In [374]:
entries.head()

,institution,date,entrees,day,month,year,week
0,joc,01/01/2019,0,01,01,2019,1
1,joc,01/11/2018,35962,01,11,2018,44
2,joc,01/12/2018,18081,01,12,2018,48
3,joc,02/01/2019,46300,02,01,2019,1
4,joc,02/11/2018,41984,02,11,2018,44


## make the dataset smaller

Restrain to january 2019

In [375]:
#entries = entries[entries.date.str.contains("2019")]

## compute cumulative data

In [376]:
entries = entries.sort_values(['institution','year', 'month','day'], ascending=True)

In [377]:
entries.reset_index(drop=True)
entries['sumentrees'] = entries.groupby(['institution'])['entrees'].cumsum()

In [378]:
entries

,institution,date,entrees,day,month,year,week,sumentrees
1,joc,01/11/2018,35962,01,11,2018,44,35962
4,joc,02/11/2018,41984,02,11,2018,44,77946
7,joc,03/11/2018,29930,03,11,2018,44,107876
10,joc,04/11/2018,29092,04,11,2018,44,136968
13,joc,05/11/2018,22184,05,11,2018,45,159152
16,joc,06/11/2018,0,06,11,2018,45,159152
19,joc,07/11/2018,26093,07,11,2018,45,185245
22,joc,08/11/2018,19305,08,11,2018,45,204550
25,joc,09/11/2018,27565,09,11,2018,45,232115
28,joc,10/11/2018,29326,10,11,2018,45,261441


In [379]:
entries.to_csv("3_months.csv", sep=',', encoding='utf-8', index=False)